# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Date List Created
Record num : 123365
Wall time: 13.4 s


In [3]:
%%time
data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
data['df']['DateTime_frmt']   = pd.to_datetime(data["df"]['DateTime'], format="%Y%m%d %H:%M:%S")
data['df_ohlc'] = data['df'][['DateTime_frmt', 'tick']]

Wall time: 32.8 ms


In [4]:
%%time
data = get_ohlc(data)

Wall time: 23.9 ms


In [5]:
data['df_ohlc']['cdl_hammer'] = talib.CDLHAMMER(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])
data['df_ohlc']['cdl_engulfing'] = talib.CDLENGULFING(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])
data['df_ohlc']['cdl_shootingstar'] = talib.CDLSHOOTINGSTAR(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])

### Merge with DF to visualize Data

In [6]:
data = merge_ohlc_data(data)

100%|████████████████████████████████████████| 288/288 [00:03<00:00, 80.89it/s]


In [7]:
data['df']['cdl_hammer'] = data['df']['cdl_hammer'].replace({0:np.nan})
data['df']['cdl_hammer'] = np.where(data['df']['cdl_hammer'] == 100, data['df']['tick'], data['df']['cdl_hammer'])

In [8]:
data['df']['cdl_shootingstar'] = data['df']['cdl_shootingstar'].replace({0:np.nan})
data['df']['cdl_shootingstar'] = np.where(data['df']['cdl_shootingstar'] == -100, data['df']['tick'], data['df']['cdl_shootingstar'])

In [9]:
data['df']['cdl_engulfing_up'] = data['df']['cdl_engulfing'][data['df']['cdl_engulfing'] == 100]
data['df']['cdl_engulfing_down'] = data['df']['cdl_engulfing'][data['df']['cdl_engulfing'] == -100]

data['df']['cdl_engulfing_up'] = data['df']['cdl_engulfing_up'].replace({0:np.nan})
data['df']['cdl_engulfing_up'] = np.where(data['df']['cdl_engulfing_up'] == 100, data['df']['tick'], data['df']['cdl_engulfing_up'])

data['df']['cdl_engulfing_down'] = data['df']['cdl_engulfing_down'].replace({0:np.nan})
data['df']['cdl_engulfing_down'] = np.where(data['df']['cdl_engulfing_down'] == -100, data['df']['tick'], data['df']['cdl_engulfing_down'])

del data['df']['cdl_engulfing'] 

In [10]:
data['df']['sema'] = talib.EMA(data['df']['tick'], timeperiod=500)
data['df']['dema'] = talib.DEMA(data['df']['tick'], timeperiod=5000)
data['df']['mom'] = talib.MOM(data['df']['tick'], timeperiod=5000)

### Plot

In [11]:
plot_graph(data)

sys.exit()

data['identifier_type'] = 'cdl_hammer'
data['identifier_val'] = 100
data['test_type'] = 'up'

data = calculate_total_profit(data)

data['identifier_type'] = 'cdl_engulfing'
data['identifier_val'] = 100
data['test_type'] = 'up'
data = calculate_total_profit(data)

data['identifier_type'] = 'cdl_engulfing'
data['identifier_val'] = -100
data['test_type'] = 'down'

data = calculate_total_profit(data)

data['df_ohlc'].to_csv('data/df_ohlc.csv')
data['df'].to_csv('data/df.csv')